In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import glob
import json
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
import re
# test test test

def check_list_contained(A, B):
  # convert list A to string
    A_str = ' '.join(map(str, A))
    # convert list B to string
    B_str = ' '.join(map(str, B))
    # find all instances of A within B
    instances = re.findall(A_str, B_str)

    # return True if any instances were found, False otherwise
    return len(instances) > 0

In [10]:
# CHANGE HERE TO IMPORT OR SAVE
saved = False

if saved:
    data = np.load('processed_data.npz')
    xData = data['xData']
    yData = data['yData']
else:

In [19]:
# Path to the folder containing the JSON files

# Load the DataFrames outside of the function
isosignsDF = pd.read_excel('datasignlist.xlsx', sheet_name='IsoSigns', usecols="B:D")
singlewordDF = pd.read_excel('datasignlist.xlsx', sheet_name='SingleWord', usecols="B:D")

def get_gloss_label(file_name, isosignsDF, singlewordDF):
    # Check if the file name starts with a digit to determine which DataFrame to use
    # file_name = file_name.split("/")[-1]
    if file_name[0].isdigit():
        # Extract Video_ID from the filename
        video_id = file_name.split('_')[0]
        # Retrieve the row from the isosigns DataFrame based on Video_ID
        gloss_row = isosignsDF.loc[isosignsDF['Video_ID'] == int(video_id)]
    else:
        # Extract Video_Clip_Name from the filename
        video_clip_name = '_'.join(file_name.split('_')[:3])
        # Retrieve the row from the singleword DataFrame based on Video_Clip_Name
        gloss_row = singlewordDF.loc[singlewordDF['Video_Clip_Name'] == video_clip_name]

    # Check if a match was found and return the Gloss value
    if not gloss_row.empty:
        return gloss_row['Gloss'].values[0]  # Note the capitalization of 'Gloss'
    else:
        return None

def process_file(file):
    print(f"Processing {file}")
    largestFrames = 0
    xData = []
    yData = []

    with open(file, 'r') as f:
        d = f.read()
        vidData = json.loads(d)

        data = []
        for frame in vidData:
            if 'data' not in frame.keys():
                continue

            fData = pd.DataFrame(frame['data'])
            data.append(fData)

        if len(data) == 0:
            return None, None, largestFrames

        combinedDF = pd.concat(data, ignore_index=True)
        reshapedData = combinedDF.values.reshape((len(data), 75, 4))
        largestFrames = max(largestFrames, len(data))

        word = get_gloss_label(file, isosignsDF, singlewordDF)

        # Handle missing gloss
        if word is None:
            print(f"Warning: No gloss label found for {file}")
            return None, None, largestFrames

        xData.append(reshapedData)
        yData.append(word)

    return xData, yData, largestFrames
    
files = glob.glob('json_keypoints/*.json')
count = 0

xData = []
yData = []
largestFrames = 0

start = time.time()
count = 0

# Create a ProcessPoolExecutor for parallel processing
with ProcessPoolExecutor(max_workers=1) as executor:
    # Use a list comprehension to submit the processing of each file
    futures = {executor.submit(process_file, file_name): file_name for file_name in files}

    for future in futures:
        reshaped_data, label, maxFrames = future.result()
        if reshaped_data is not None and label is not None:
            xData.extend(reshaped_data)  # Extend instead of append to flatten the structure
            yData.append(label)
            largestFrames = max(largestFrames, maxFrames)

        if count%500 == 0:
            print(count)
            print(time.time()-start)
        count += 1

xData = np.array(xData, dtype=object)
yData = np.array(yData)

# x_train, x_test, y_train, y_test = train_test_split(xData, yData, test_size=0.2, stratify=yData, random_state=42)

# print("Train samples:", len(x_train), "Test samples:", len(x_test))
print("Largest frames:", largestFrames)
print("Time taken:", time.time() - start)

np.savez_compressed('processed_data.npz', xData=xData, yData=yData)


Process SpawnProcess-12:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_file' on <module '__main__' (built-in)>


BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore